In [1]:
import zipfile
import os

# Path to your uploaded ZIP file (update if your filename is different)
zip_path = "/content/Image_preprocessing(train_test_10K).zip"  # or change this to match your uploaded filename

# Destination folder (where files will be extracted)
extract_to = "/content/Image_preprocessing(train_test_10K)"

# Create folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files extracted successfully to:", extract_to)

# List extracted contents
for root, dirs, files in os.walk(extract_to):
    level = root.replace(extract_to, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


Files extracted successfully to: /content/Image_preprocessing(train_test_10K)
Image_preprocessing(train_test_10K)/
  Image_preprocessing(train_test_10K)/
    image_embeddings_train.npy
    Train_Test(10K)_image_preprocessing.ipynb
    image_embeddings_test.npy


In [2]:
import zipfile
import os

# Path to your uploaded ZIP file (update if your filename is different)
zip_path = "/content/artifacts_10k.zip"  # or change this to match your uploaded filename

# Destination folder (where files will be extracted)
extract_to = "/content/artifacts_10k"

# Create folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files extracted successfully to:", extract_to)

# List extracted contents
for root, dirs, files in os.walk(extract_to):
    level = root.replace(extract_to, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


Files extracted successfully to: /content/artifacts_10k
artifacts_10k/
  artifacts_10k/
    image_embeddings_train.npy
    image_embeddings_test.npy


In [4]:
import zipfile
import os

# Path to your uploaded ZIP file (update if your filename is different)
zip_path = "/content/student_resource.zip"  # or change this to match your uploaded filename

# Destination folder (where files will be extracted)
extract_to = "/content/amazon_ml_challenge"

# Create folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files extracted successfully to:", extract_to)

# List extracted contents
for root, dirs, files in os.walk(extract_to):
    level = root.replace(extract_to, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

Files extracted successfully to: /content/amazon_ml_challenge
amazon_ml_challenge/
  student_resource/
    .DS_Store
    Documentation_template.md
    sample_code.py
    README.md
    dataset/
      sample_test.csv
      train.csv
      sample_test_out.csv
      test.csv
    src/
      utils.py
      example.ipynb
      __pycache__/
        utils.cpython-37.pyc


In [12]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import os
import joblib

# CONFIG
DATA_DIR = "/content/amazon_ml_challenge/dataset"
ARTIFACTS_DIR = "/content/amazon_ml_challenge/artifacts_10k"
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# Load Data
train_path = os.path.join("/content/amazon_ml_challenge/student_resource/dataset/train.csv")
test_path = os.path.join("/content/amazon_ml_challenge/student_resource/dataset/test.csv")

train = pd.read_csv(train_path).head(10000)
test = pd.read_csv(test_path).head(10000)

# Helper Regex Functions
def extract_pack_count(text):
    text = str(text).lower()
    m = re.search(r'(\d+)\s*(pack|count|ct|pcs|x|×)', text)
    return int(m.group(1)) if m else 1

def extract_weight_grams(text):
    text = str(text).lower()
    if match := re.search(r'(\d+\.?\d*)\s*(kg|g|ml|l|oz)', text):
        val, unit = float(match.group(1)), match.group(2)
        if unit == 'kg': return val * 1000
        if unit == 'l': return val * 1000
        if unit == 'oz': return val * 28.35
        return val
    return np.nan

def clean_text(text):
    text = str(text).lower()
    text = re.sub(f"[{string.punctuation}]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def text_stats(text):
    t = str(text)
    return {
        "title_len": len(t),
        "catalog_len": len(t.split()),
        "unique_words": len(set(t.split())),
        "num_digits": sum(c.isdigit() for c in t),
        "num_caps": sum(c.isupper() for c in t)
    }

# Parse Fields

def parse_catalog(df):
    df["pack_count"] = df["catalog_content"].apply(extract_pack_count)
    df["weight_grams"] = df["catalog_content"].apply(extract_weight_grams)

    stats = df["catalog_content"].apply(text_stats)
    stats_df = pd.DataFrame(list(stats))

    df = pd.concat([df, stats_df], axis=1)
    df["clean_text"] = df["catalog_content"].apply(clean_text)
    return df

train = parse_catalog(train)
test = parse_catalog(test)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    stop_words='english'
)

X_train_tfidf = vectorizer.fit_transform(train["clean_text"])
X_test_tfidf = vectorizer.transform(test["clean_text"])

# Save Artifacts
numeric_cols = ["sample_id", "pack_count", "weight_grams", "title_len",
                "catalog_len", "unique_words", "num_digits", "num_caps"]

train[numeric_cols + ["price"]].to_csv(f"{ARTIFACTS_DIR}/train_numeric.csv", index=False)
test[numeric_cols].to_csv(f"{ARTIFACTS_DIR}/test_numeric.csv", index=False)

sparse.save_npz(f"{ARTIFACTS_DIR}/X_train_tfidf.npz", X_train_tfidf)
sparse.save_npz(f"{ARTIFACTS_DIR}/X_test_tfidf.npz", X_test_tfidf)
joblib.dump(vectorizer, f"{ARTIFACTS_DIR}/tfidf_vectorizer.pkl")

parsed_fields = train[["sample_id", "pack_count", "weight_grams", "catalog_content"]]
parsed_fields.to_csv(f"{ARTIFACTS_DIR}/parsed_fields.csv", index=False)

print("Data cleaning & parsing completed!")
print("TF-IDF shape:", X_train_tfidf.shape)
print("Numeric feature sample:\n", train[numeric_cols].head())

Data cleaning & parsing completed!
TF-IDF shape: (10000, 10000)
Numeric feature sample:
    sample_id  pack_count  weight_grams  title_len  catalog_len  unique_words  \
0      33127           1           NaN         91           18            18   
1     198967           1           NaN        511           80            58   
2     261251           1        0.0000        328           59            47   
3      55858           1      318.9375       1318          211           142   
4     292686           1           NaN        155           28            18   

   num_digits  num_caps  
0           6        14  
1          17        39  
2          12        29  
3          17        60  
4          13        15  


In [18]:
import pandas as pd
import numpy as np
import os

ARTIFACTS_DIR = "/content/amazon_ml_challenge/artifacts_10k"

# Load the numeric features
train_numeric = pd.read_csv(f"{ARTIFACTS_DIR}/train_numeric.csv")
test_numeric = pd.read_csv(f"{ARTIFACTS_DIR}/test_numeric.csv")

# Fill missing weight with -1
train_numeric["weight_grams"] = train_numeric["weight_grams"].fillna(-1)
test_numeric["weight_grams"] = test_numeric["weight_grams"].fillna(-1)


In [20]:
# Cross features
train_numeric["pack_title_interaction"] = train_numeric["pack_count"] * train_numeric["title_len"]
test_numeric["pack_title_interaction"] = test_numeric["pack_count"] * test_numeric["title_len"]

# Basic char/word ratio
train_numeric["char_per_word"] = train_numeric["title_len"] / (train_numeric["catalog_len"]+1)
test_numeric["char_per_word"] = test_numeric["title_len"] / (test_numeric["catalog_len"]+1)

# Save updated numeric features
train_numeric.to_csv(f"{ARTIFACTS_DIR}/train_numeric_enhanced.csv", index=False)
test_numeric.to_csv(f"{ARTIFACTS_DIR}/test_numeric_enhanced.csv", index=False)

print("Enhanced numeric features added!")
print("Train numeric sample:\n", train_numeric.head())
print("Train numeric sample:\n", test_numeric.head())


Enhanced numeric features added!
Train numeric sample:
    sample_id  pack_count  weight_grams  title_len  catalog_len  unique_words  \
0      33127           1       -1.0000         91           18            18   
1     198967           1       -1.0000        511           80            58   
2     261251           1        0.0000        328           59            47   
3      55858           1      318.9375       1318          211           142   
4     292686           1       -1.0000        155           28            18   

   num_digits  num_caps  price  pack_title_interaction  char_per_word  
0           6        14   4.89                      91       4.789474  
1          17        39  13.12                     511       6.308642  
2          12        29   1.97                     328       5.466667  
3          17        60  30.34                    1318       6.216981  
4          13        15  66.49                     155       5.344828  
Train numeric sample:
    sampl

In [21]:
train_numeric_enhanced=  pd.read_csv(f"{ARTIFACTS_DIR}/train_numeric_enhanced.csv")
test_numeric_enhanced=  pd.read_csv(f"{ARTIFACTS_DIR}/test_numeric_enhanced.csv")
print("Train numeric enhanced sample:\n", train_numeric_enhanced.head())
print("Test numeric enhanced sample:\n", test_numeric_enhanced.head())


Train numeric enhanced sample:
    sample_id  pack_count  weight_grams  title_len  catalog_len  unique_words  \
0      33127           1       -1.0000         91           18            18   
1     198967           1       -1.0000        511           80            58   
2     261251           1        0.0000        328           59            47   
3      55858           1      318.9375       1318          211           142   
4     292686           1       -1.0000        155           28            18   

   num_digits  num_caps  price  pack_title_interaction  char_per_word  
0           6        14   4.89                      91       4.789474  
1          17        39  13.12                     511       6.308642  
2          12        29   1.97                     328       5.466667  
3          17        60  30.34                    1318       6.216981  
4          13        15  66.49                     155       5.344828  
Test numeric enhanced sample:
    sample_id  pack_count

In [22]:
import numpy as np
import pandas as pd
from scipy.sparse import hstack, csr_matrix, save_npz, load_npz
import os

# Adjust path if different
ARTIFACTS_DIR = "/content/amazon_ml_challenge/artifacts_10k"


In [23]:
# === Load TF-IDF text features ===
X_train_tfidf = load_npz(f"{ARTIFACTS_DIR}/X_train_tfidf.npz")
X_test_tfidf = load_npz(f"{ARTIFACTS_DIR}/X_test_tfidf.npz")
print("TF-IDF:", X_train_tfidf.shape, X_test_tfidf.shape)


TF-IDF: (10000, 10000) (10000, 10000)


In [24]:
# Separate target variable
y_train = train_numeric["price"].values
train_numeric = train_numeric.drop(columns=["sample_id", "price"], errors='ignore')
test_numeric = test_numeric.drop(columns=["sample_id"], errors='ignore')

# Convert numeric features to sparse matrices
X_train_num = csr_matrix(train_numeric.values)
X_test_num = csr_matrix(test_numeric.values)

print("Numeric:", X_train_num.shape, X_test_num.shape)

Numeric: (10000, 9) (10000, 9)


In [28]:
# === Load image embeddings ===
img_train = np.load("/content/Image_preprocessing(train_test_10K)/Image_preprocessing(train_test_10K)/image_embeddings_train.npy")
img_test = np.load("/content/Image_preprocessing(train_test_10K)/Image_preprocessing(train_test_10K)/image_embeddings_test.npy")

# Convert image arrays to sparse
X_train_img = csr_matrix(img_train)
X_test_img = csr_matrix(img_test)
print("Image:", X_train_img.shape, X_test_img.shape)

# === Combine all sources ===
X_train_combined = hstack([X_train_tfidf, X_train_num, X_train_img]).tocsr()
X_test_combined = hstack([X_test_tfidf, X_test_num, X_test_img]).tocsr()

print("\nCombined feature shapes:")
print("Train:", X_train_combined.shape)
print("Test:", X_test_combined.shape)


Image: (10000, 2048) (10000, 2048)

Combined feature shapes:
Train: (10000, 12057)
Test: (10000, 12057)


In [29]:
# === Save combined datasets ===
save_npz(f"{ARTIFACTS_DIR}/X_train_combined.npz", X_train_combined)
save_npz(f"{ARTIFACTS_DIR}/X_test_combined.npz", X_test_combined)
np.save(f"{ARTIFACTS_DIR}/y_train.npy", y_train)

In [30]:
import numpy as np
import pandas as pd
import os

ARTIFACTS_DIR = "/content/amazon_ml_challenge/artifacts_10k"

# Load raw target
y_train = np.load(f"{ARTIFACTS_DIR}/y_train.npy")

# Apply log1p transform (natural log(1 + price))
y_train_log = np.log1p(y_train)

# Save transformed target
np.save(f"{ARTIFACTS_DIR}/y_train_log.npy", y_train_log)

print("Target transformation complete!")
print("Raw price sample:", y_train[:5])
print("Log-transformed sample:", y_train_log[:5])
print(f"Mean raw price: {y_train.mean():.4f}, Mean log price: {y_train_log.mean():.4f}")


Target transformation complete!
Raw price sample: [ 4.89 13.12  1.97 30.34 66.49]
Log-transformed sample: [1.773256   2.64759223 1.08856195 3.44489524 4.21197944]
Mean raw price: 23.6783, Mean log price: 2.7387


In [31]:
def smape(y_true, y_pred):
    """
    Symmetric Mean Absolute Percentage Error (SMAPE)
    Formula:
        SMAPE = mean( |y_true - y_pred| / ((|y_true| + |y_pred|) / 2) )
    Returns value in % (0–200).
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    denom = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred)
    smape_val = np.mean(np.where(denom == 0, 0, diff / denom)) * 100
    return smape_val

# Test the function
print("SMAPE %:", smape([100], [120]), "%")


SMAPE %: 18.181818181818183 %
